In [1]:
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install scikit-learn
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 69.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', 200)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X_train = pd.read_csv("/content/drive/MyDrive/aviahack/X_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/aviahack/y_train.csv")

In [5]:
X_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47794 entries, 0 to 47793
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   engine_id        47794 non-null  object 
 1   aircraft_id      47794 non-null  object 
 2   flight_datetime  47794 non-null  object 
 3   flight_phase     47794 non-null  object 
 4   engine_position  47794 non-null  int64  
 5   n1_modifier      47794 non-null  float64
 6   number_blades    47794 non-null  float64
 7   engine_family    47794 non-null  object 
 8   engine_type      47794 non-null  object 
 9   manufacturer     47794 non-null  object 
 10  ZHPTAC           11957 non-null  float64
 11  ZLPTAC           11957 non-null  float64
 12  ZPCN12           47794 non-null  float64
 13  ZPCN25           47794 non-null  float64
 14  ZPHSF            13336 non-null  float64
 15  ZPHSR            13336 non-null  float64
 16  ZPN12R           18382 non-null  float64
 17  ZPOIL       

In [6]:
corr = X_train.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,engine_position,n1_modifier,number_blades,ZHPTAC,ZLPTAC,ZPCN12,ZPCN25,ZPHSF,ZPHSR,ZPN12R,ZPOIL,ZPS3,ZT1AB,ZT3,ZT49,ZTAMB,ZTLA,ZTNAC,ZTOIL,ZVB1F,ZVB1R,ZVB2F,ZVB2R,ZVSV,ZWF36,IHPSOV,AGW,CAS,IAI,IVS12,SAT,ZALT,ZT1A,ZVIAS,ZWBP1,ZWBP1_8E,ZWBP2,ZWBP2_8E,ZXM,IBE,IBP,IAIE
engine_position,1.00,-0.00,-0.01,-0.01,-0.01,-0.00,0.01,-0.01,0.02,-0.01,-0.01,0.01,0.01,0.01,-0.00,0.01,0.01,0.04,-0.01,-0.04,0.15,0.06,-0.01,-0.01,-0.00,nan,-0.00,-0.01,0.01,0.01,-0.01,-0.00,0.00,-0.01,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.02,0.00
n1_modifier,-0.00,1.00,0.67,-0.01,0.22,-0.07,0.35,0.13,-0.21,nan,-0.64,nan,nan,nan,-0.52,nan,nan,-0.34,0.16,-0.17,-0.09,0.31,-0.58,0.64,-0.17,nan,-0.09,0.42,-0.22,-0.01,-0.04,0.26,-0.19,nan,-0.08,nan,-0.08,nan,0.26,-0.18,0.05,0.06
number_blades,-0.01,0.67,1.00,nan,nan,-0.25,0.11,nan,nan,nan,-0.43,nan,nan,nan,-0.40,nan,nan,nan,-0.08,-0.03,0.06,0.36,-0.36,0.72,-0.06,nan,0.03,0.68,0.06,-0.02,nan,0.29,-0.24,nan,nan,nan,nan,nan,0.29,-0.20,-0.09,-0.07
ZHPTAC,-0.01,-0.01,nan,1.00,0.49,0.67,0.39,-0.00,-0.02,nan,0.23,nan,nan,nan,0.27,nan,nan,-0.14,-0.16,0.03,0.07,-0.02,-0.12,-0.72,0.34,nan,0.03,-0.12,nan,0.01,nan,0.59,-0.40,nan,0.00,nan,0.00,nan,0.69,-0.01,nan,nan
ZLPTAC,-0.01,0.22,nan,0.49,1.00,0.16,0.09,0.02,-0.04,nan,-0.19,nan,nan,nan,-0.13,nan,nan,-0.09,0.19,-0.01,-0.05,0.13,-0.01,-0.14,-0.30,nan,-0.29,-0.21,nan,-0.03,nan,0.42,-0.08,nan,-0.03,nan,-0.03,nan,0.35,0.01,nan,nan
ZPCN12,-0.00,-0.07,-0.25,0.67,0.16,1.00,0.77,-0.02,-0.01,0.54,-0.02,-0.07,0.16,0.50,0.46,0.05,0.21,0.23,0.03,0.08,0.06,-0.13,-0.15,-0.61,0.12,nan,-0.02,-0.45,-0.26,-0.55,0.10,-0.07,0.17,0.09,0.01,-0.12,0.01,-0.10,-0.06,-0.04,0.24,-0.02
ZPCN25,0.01,0.35,0.11,0.39,0.09,0.77,1.00,-0.03,-0.00,-0.41,-0.43,0.74,0.91,1.00,0.11,0.85,0.83,0.44,0.02,-0.12,0.07,-0.00,-0.62,0.52,0.22,nan,-0.09,-0.40,-0.34,-0.58,0.15,-0.13,0.26,-0.71,-0.01,-0.12,-0.01,-0.12,-0.11,-0.15,0.18,0.06
ZPHSF,-0.01,0.13,nan,-0.00,0.02,-0.02,-0.03,1.00,0.00,nan,-0.04,nan,nan,nan,-0.05,nan,nan,0.05,0.08,-0.09,0.07,-0.11,-0.02,-0.01,-0.02,nan,0.00,0.06,-0.00,-0.01,nan,0.06,-0.04,nan,-0.01,nan,-0.01,nan,0.06,0.05,nan,0.03
ZPHSR,0.02,-0.21,nan,-0.02,-0.04,-0.01,-0.00,0.00,1.00,nan,-0.12,nan,nan,nan,-0.01,nan,nan,0.09,0.06,0.11,0.05,-0.09,0.02,0.02,0.00,nan,0.12,0.00,-0.02,-0.00,nan,0.01,-0.01,nan,0.03,nan,0.03,nan,0.01,0.02,nan,0.02
ZPN12R,-0.01,nan,nan,nan,nan,0.54,-0.41,nan,nan,1.00,-0.55,-0.81,-0.74,-0.46,-0.16,-0.80,-0.59,nan,0.64,0.36,nan,nan,0.02,-0.97,-0.78,nan,nan,nan,0.01,nan,nan,0.87,-0.75,0.79,nan,-0.00,nan,0.02,0.86,0.02,0.15,-0.18


Без дропов числовых колонок и без нормализации. Сравнение заполнения пропусков медианой,средним,модой

In [7]:
dfy_median = y_train.copy()
dfy_mean = y_train.copy()
dfy_mode = y_train.copy()
for ser in y_train:
  if y_train[ser].dtype != 'object':
    dfy_median[ser] = y_train[ser].fillna(y_train[ser].median())
    dfy_mean[ser] = y_train[ser].fillna(y_train[ser].mean())    
    dfy_mode[ser] = y_train[ser].fillna(y_train[ser].mode()[0])    

In [8]:
df_median = X_train.copy()
df_mean = X_train.copy()
df_mode = X_train.copy()
for ser in X_train:
  if X_train[ser].dtype != 'object':
    df_median[ser] = X_train[ser].fillna(X_train[ser].median())
    df_mean[ser] = X_train[ser].fillna(X_train[ser].mean())    
    df_mode[ser] = X_train[ser].fillna(X_train[ser].mode()[0])

In [9]:
for frame in [df_median,df_mean,df_mode]:
  frame = frame.drop(columns=[
    "engine_id", "aircraft_id", "flight_datetime",
    "flight_phase", "engine_family", "engine_type", "manufacturer",
    "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"],inplace=True)
for frame in [dfy_median,dfy_mean,dfy_mode]:
  frame = frame.drop(columns=["engine_id", "flight_phase", "flight_datetime"],inplace=True)

In [10]:
def get_score_by_parameter(predict_field,df,dfy):
    
    X = df
    y = dfy[predict_field]

    
    Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=100, verbose=False)
    model.fit(Xm_train, ym_train)
    catboost_predicted = model.predict(Xm_test)
    catboost_score = mean_squared_error(ym_test, catboost_predicted)
    return catboost_score

In [ ]:

scores = pd.DataFrame(columns=["parameter", "rmse_median", "rmse_mode", "rmse_median"])
output_parameters = y_train.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])

for param in tqdm(output_parameters):
    try:
        score_median = get_score_by_parameter(param,df_median,dfy_median)
        score_mean = get_score_by_parameter(param,df_mean,dfy_mean)
        score_mode = get_score_by_parameter(param,df_mode,dfy_mode)
        scores.loc[len(scores)] = [param, score_median, score_mean, score_mode]
    except Exception as error:
      print(error,'not good')
      continue



 73%|███████▎  | 22/30 [01:45<00:28,  3.51s/it]

catboost/libs/metrics/metric.cpp:6376: All train targets are equal not good


 93%|█████████▎| 28/30 [02:13<00:08,  4.49s/it]

In [ ]:
scores

Без дропов числовых колонок с дропом строк целевой где наны. Сравнение заполнения пропусков иксов медианой,средним,модой

In [ ]:
X_train = pd.read_csv("/content/drive/MyDrive/aviahack/X_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/aviahack/y_train.csv")

In [ ]:
dfy = y_train.copy()

In [ ]:
df_median = X_train.copy()
df_mean = X_train.copy()
df_mode = X_train.copy()
for ser in X_train:
  if X_train[ser].dtype != 'object':
    df_median[ser] = X_train[ser].fillna(X_train[ser].median())
    df_mean[ser] = X_train[ser].fillna(X_train[ser].mean())    
    df_mode[ser] = X_train[ser].fillna(X_train[ser].mode()[0])

In [ ]:
def get_score_by_parameter(predict_field,df,dfy):
    df = pd.merge(df, dfy, on=["engine_id", "flight_datetime", "flight_phase"])
    df = df[df[predict_field].notna()]
    
    X = df.drop(columns=[predict_field])
    X = X.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    y = df[predict_field]

    Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=100, verbose=False)
    model.fit(Xm_train, ym_train)
    catboost_predicted = model.predict(Xm_test)
    catboost_score = mean_squared_error(ym_test, catboost_predicted)
    return catboost_score

In [ ]:
scores = pd.DataFrame(columns=["parameter", "rmse_median", "rmse_mean", "rmse_mode"])
output_parameters = y_train.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])

for param in tqdm(output_parameters):
    try:
        score_median = get_score_by_parameter(param,df_median,dfy)
        score_mean = get_score_by_parameter(param,df_mean,dfy)
        score_mode = get_score_by_parameter(param,df_mode,dfy)

        scores.loc[len(scores)] = [param, score_median, score_mean, score_mode]
    except Exception as error:
      print(error,'not good')
      continue



In [ ]:
scores

ВАЛИД

In [ ]:
X_valid = pd.read_csv('/content/drive/MyDrive/aviahack/X_valid.csv')

In [ ]:
X_train = pd.read_csv("/content/drive/MyDrive/aviahack/X_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/aviahack/y_train.csv")

In [ ]:
dfy = y_train.copy()

In [ ]:
df_median = X_train.copy()
df_mean = X_train.copy()
df_mode = X_train.copy()
for ser in X_train:
  if X_train[ser].dtype != 'object':
    df_median[ser] = X_train[ser].fillna(X_train[ser].median())
    df_mean[ser] = X_train[ser].fillna(X_train[ser].mean())    
    df_mode[ser] = X_train[ser].fillna(X_train[ser].mode()[0])

In [ ]:
def get_result(predict_field,df,dfy,dfvalid):


    
    X = df
    y = dfy[predict_field]
    
    X = X.drop(axis=1, index=dfy[dfy[predict_field].isna()].index)
    # X.info()
    y = y.drop(axis=1, index=dfy[dfy[predict_field].isna()].index)
    # y

    X = X.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    dfvalid = dfvalid.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    dfvalid[ser] = dfvalid[ser].fillna(dfvalid[ser].mean())   
    
    Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=300, verbose=False,depth=10)
    model.fit(Xm_train, ym_train)
    catboost_predicted = model.predict(Xm_test)
    catboost_score = mean_squared_error(ym_test, catboost_predicted)
    catboost_predicted = model.predict(dfvalid)
    print(catboost_score)
    return catboost_predicted

In [ ]:
output_parameters = y_train.columns.drop(['flight_datetime', 'flight_phase', 'engine_id']).values
y_valid = pd.DataFrame(columns=[output_parameters],index=range(0,X_valid.shape[0]))

for param in tqdm(output_parameters):
    try:
        pred = get_result(param,df_mean,dfy,X_valid)

        y_valid[param.strip(',()')] = pd.Series(pred)
    except Exception as error:
      print(error,'not good')
      continue


  3%|▎         | 1/30 [00:31<15:25, 31.92s/it]

0.000407114286639515


  7%|▋         | 2/30 [00:58<13:30, 28.96s/it]

6.147517135572378


 10%|█         | 3/30 [01:14<10:12, 22.69s/it]

...

0.003163783004527109


 97%|█████████▋| 29/30 [10:49<00:20, 20.79s/it]

5.8479681725108055


100%|██████████| 30/30 [11:18<00:00, 22.61s/it]

16847.337904991993


In [ ]:
y_valid = y_valid.fillna(0)

In [ ]:
y_valid = X_valid[['flight_datetime', 'flight_phase', 'engine_id']].merge(y_valid,left_index = True,right_index=True)

In [ ]:
y_valid.to_csv('/content/drive/MyDrive/y_valid.csv')